In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)



In [2]:

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 3x3 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Convolutional Layer #2
  # Computes 64 features using a 3x3 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 28, 28, 64]
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 64]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  pool = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)


  # Add dropout operation; 0.25 probability that element will be kept
  dropout1 = tf.layers.dropout(
      inputs=pool, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  flat = tf.reshape(dropout1, [-1, 14 * 14 * 64])

  # Dense Layer
  # Densely connected layer with 128 neurons
  # Input Tensor Shape: [batch_size, 14 * 14 * 64]
  # Output Tensor Shape: [batch_size, 128]
  dense = tf.layers.dense(inputs=flat, units=128, activation=tf.nn.relu)

  # Add dropout operation; 0.25 probability that element will be kept
  dropout2 = tf.layers.dropout(
      inputs=dense, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 128]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout2, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:


def main():
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=1000)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
  input_fn=train_input_fn,
  steps=20000)

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)

main()
# if __name__ == "__main__":
#   tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a16d8b490>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_model', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_model/model.ckpt.
INFO:tensorflow:loss = 2.30354, step = 1
INFO:tensorflow:global_step/sec: 4.5551
INFO:tensorflow:loss = 2.2918

INFO:tensorflow:loss = 0.3501, step = 7301 (23.029 sec)
INFO:tensorflow:global_step/sec: 4.16572
INFO:tensorflow:loss = 0.375219, step = 7401 (24.005 sec)
INFO:tensorflow:global_step/sec: 4.32829
INFO:tensorflow:loss = 0.21314, step = 7501 (23.104 sec)
INFO:tensorflow:global_step/sec: 4.30371
INFO:tensorflow:loss = 0.205585, step = 7601 (23.236 sec)
INFO:tensorflow:global_step/sec: 4.236
INFO:tensorflow:loss = 0.292881, step = 7701 (23.607 sec)
INFO:tensorflow:global_step/sec: 4.27738
INFO:tensorflow:loss = 0.244786, step = 7801 (23.379 sec)
INFO:tensorflow:Saving checkpoints for 7856 into /tmp/mnist_model/model.ckpt.
INFO:tensorflow:global_step/sec: 4.17066
INFO:tensorflow:loss = 0.23974, step = 7901 (23.977 sec)
INFO:tensorflow:global_step/sec: 4.24449
INFO:tensorflow:loss = 0.161826, step = 8001 (23.560 sec)
INFO:tensorflow:global_step/sec: 4.23009
INFO:tensorflow:loss = 0.510892, step = 8101 (23.640 sec)
INFO:tensorflow:global_step/sec: 4.26469
INFO:tensorflow:loss = 0.423349, step

INFO:tensorflow:loss = 0.0998782, step = 15301 (21.959 sec)
INFO:tensorflow:global_step/sec: 4.38032
INFO:tensorflow:loss = 0.218186, step = 15401 (22.837 sec)
INFO:tensorflow:global_step/sec: 4.62766
INFO:tensorflow:loss = 0.230301, step = 15501 (21.602 sec)
INFO:tensorflow:global_step/sec: 4.58606
INFO:tensorflow:loss = 0.203125, step = 15601 (21.805 sec)
INFO:tensorflow:Saving checkpoints for 15655 into /tmp/mnist_model/model.ckpt.
INFO:tensorflow:global_step/sec: 4.6479
INFO:tensorflow:loss = 0.108997, step = 15701 (21.515 sec)
INFO:tensorflow:global_step/sec: 4.09119
INFO:tensorflow:loss = 0.283512, step = 15801 (24.443 sec)
INFO:tensorflow:global_step/sec: 4.52129
INFO:tensorflow:loss = 0.204211, step = 15901 (22.118 sec)
INFO:tensorflow:global_step/sec: 4.48603
INFO:tensorflow:loss = 0.195977, step = 16001 (22.292 sec)
INFO:tensorflow:global_step/sec: 4.27892
INFO:tensorflow:loss = 0.301479, step = 16101 (23.370 sec)
INFO:tensorflow:global_step/sec: 4.26946
INFO:tensorflow:loss 